# Spatial join - EUBUCCO and urban audit data

In [1]:
import psycopg2
from sshtunnel import SSHTunnelForwarder
import geopandas as gpd
import pandas as pd
from contextlib import contextmanager
import paramiko
import io
import os
from os import listdir
import numpy as np


## Import urban audit boundaries

In [2]:
path_folder = 'C:/Users/charl/OneDrive/Bureau/EUBUCCO/'
path_FUA_OECD = path_folder + "Data/FUA_OECD/"
UA = gpd.read_file(path_folder + "Data/ref-urau-2018-01m.shp/URAU_RG_01M_2018_3035_CITIES.shp")
UA = UA.to_crs('epsg:3035')
UA.head()

,URAU_CODE,URAU_CATG,CNTR_CODE,URAU_NAME,CITY_CPTL,CITY_KERN,FUA_CODE,NUTS3_2016,FID,geometry
0,AT002C1,C,AT,Graz,NaN,NaN,AT002L3,AT221,AT002C1,"POLYGON ((4730644.819 2683971.695, 4730925.349..."
1,UK002C1,C,UK,Birmingham,NaN,UK002K1,UK002L3,UKG31,UK002C1,"POLYGON ((3527051.798 3331425.149, 3526645.686..."
2,UK003C1,C,UK,Leeds,NaN,NaN,UK003L2,UKE42,UK003C1,"POLYGON ((3578681.321 3461897.171, 3578234.525..."
3,UK004C1,C,UK,Glasgow,NaN,UK004K1,UK004L1,UKM82,UK004C1,"POLYGON ((3440186.418 3731485.487, 3440308.736..."
4,UK005C1,C,UK,Bradford,NaN,NaN,UK003L2,UKE41,UK005C1,"POLYGON ((3554102.709 3484877.913, 3554070.851..."


## Connect to server

In [3]:
with open('<ssh-key>', 'r') as f:
    key = f.read()
    pkey = paramiko.RSAKey.from_private_key(io.StringIO(key))


In [5]:
@contextmanager
def db_tunnel():
    with SSHTunnelForwarder(('194.163.151.34', 22), ssh_username='<username>', remote_bind_address=('172.30.0.4', 5432), ssh_pkey=pkey) as server:

        server.start()

        params = {
            'database': 'eubucco',
            'user': '<user>',
            'password': '<password>',
            'host': 'localhost',
            'port': server.local_bind_port,
            'connect_timeout': 10,
        }

        conn = psycopg2.connect(**params)
        conn.set_session(readonly=True)

        yield conn


## Retrieve data - spatial join

In [12]:
with db_tunnel() as conn:
    for i in range(550, 938):
        print(UA.iloc[i].URAU_CODE)
        boundary = UA.iloc[i].geometry
        #boundary = 'POLYGON ((3762754.4419298171997070 2890991.8044788637198508, 3764095.9635624778456986 2888187.3515811231918633, 3761960.7702519670128822 2888983.4717874415218830, 3761945.1074829874560237 2889266.3062656684778631, 3761744.4274468249641359 2890595.5182541240938008, 3762754.4419298171997070 2890991.8044788637198508))'
        id_code = UA.iloc[i].URAU_CODE
        sql = f"""
            select bu.id, bu.id_source, bc.name as country, br.name as region, b.name as city, bu.height, bu.age, bu.type, bu.type_source, bu.geometry
                from data_building as bu
                    RIGHT OUTER JOIN data_country bc on bu.country_id = bc.id
                    RIGHT OUTER JOIN data_region br on br.id = bu.region_id
                    RIGHT OUTER JOIN data_city b on b.id = bu.city_id
                WHERE ST_Within( bu.geometry, 'SRID=3035;{boundary}');
        """

        result = gpd.GeoDataFrame.from_postgis(
            sql=sql,
            con=conn,
            geom_col='geometry',
            index_col='id',
        )

        #results.append(result)
        result.to_file(
            path_folder + "Data/EUBUCCO_by_UA/" + id_code + '.shp')


ES553C1
ES554C1
ES555C1
ES556C1
ES557C1
ES558C1
HR001C1
HR002C1
HR003C1
HR004C1
HR005C1
HR006C1
HR007C1
PT006C1
IT025C1
IT062C1
IT063C1
IT519C1
IT520C1
IT064C1
IT065C1
IT066C1
IT521C1
PT007C1
PT008C1
IT027C1
IT028C1
IT067C1
IT068C1
IT070C1
IT522C1
IT029C1
IT030C1
IT069C1
IT071C1
IT072C1
PT009C1
PT010C1
IT031C1
IT035C1
IT073C1
IT501C1
IT524C1
IT525C1
PT013C1
PT015C1
IT032C1
IT033C1
IT034C1
IT502C1
IT503C1
IT526C1
PT011C1
PT012C1
IT036C1
IT504C1
IT508C1
LV001C1
LV002C1
IT009C1
IT010C1
IT037C1
IT505C1
LV003C1
LV501C1
IT038C1
IT039C1
IT506C1
PT001C1
IT011C1
IT012C1
IT040C1
IT041C1
IT042C1
IT507C1
IT510C1
SI001C1
IT013C1
IT014C1
IT043C1
IT044C1
IT046C1
IT509C1
SI002C1
PT002C1
IT015C1
IT016C1
IT017C1
IT045C1
IT047C1
IT511C1
IT512C1
IT523C1
PT003C1
IT048C1
IT052C1
IT056C1
IT513C1
IT514C1
PT004C1
IT019C1
IT020C1
IT021C1
IT054C1
IT057C1
IT515C1
IT516C1
IT058C1
IT060C1
IT517C1
IT518C1
PT005C1
IT022C1
IT023C1
IT024C1
IT026C1
IT061C1
PT508C1
HU001C1
IT008C1
PT014C1
PT016C1
PT017C1
PT018C1
PT019C1
